Imports, file reading and part of speech tagging.

In [96]:
import nltk
import wikia

# input text
text = None
with open('/home/ddw/Downloads/hp1.txt', 'r') as f:
    text = f.read()
pos_taged = nltk.pos_tag(nltk.word_tokenize(text))

def extractEntities(ne_chunked):
    data = {}
    for entity in ne_chunked:
        if isinstance(entity, nltk.tree.Tree):
            text = " ".join([word for word, tag in entity.leaves()])
            ent = entity.label()
            data[text] = ent
        else:
            continue
    return data

wikiaEntities = {}
myWikiaEntities = {}

Named entity recognition with nltk

In [23]:
ne = nltk.ne_chunk(pos_taged, binary=False)
nltkEntities = extractEntities(ne)
nltkEntities

{"'Harry": 'PERSON',
 'AAAARGH': 'ORGANIZATION',
 'ALBUS DUMBLEDORE': 'PERSON',
 'ARE': 'ORGANIZATION',
 'Abbott': 'PERSON',
 'Abou': 'PERSON',
 'Adalbert': 'PERSON',
 'Add Norbert': 'PERSON',
 'Adrian Pucey': 'PERSON',
 'Africa': 'GPE',
 'African': 'GPE',
 'Ages': 'ORGANIZATION',
 'Agrippa': 'PERSON',
 'Ahead': 'PERSON',
 'Alberic Grunnion': 'PERSON',
 'Albus': 'PERSON',
 'Albus Dumbled': 'PERSON',
 'Albus Dumbledore': 'PERSON',
 'Alicia Spinnet': 'PERSON',
 'All Occasions': 'ORGANIZATION',
 'Alley': 'PERSON',
 'Always': 'PERSON',
 'Angelina': 'PERSON',
 'Angelina Johnson': 'PERSON',
 'Anyway': 'PERSON',
 'Are': 'PERSON',
 'Argus Filch': 'PERSON',
 'Arsenius Jigger Fantastic Beasts': 'PERSON',
 'Astronomy': 'GPE',
 'Aunt Petunia': 'PERSON',
 'Auntie Marge': 'PERSON',
 'BELIEVE': 'ORGANIZATION',
 'BOOKS': 'ORGANIZATION',
 'BOY': 'ORGANIZATION',
 'Bane': 'PERSON',
 'Barn': 'PERSON',
 'Baron': 'PERSON',
 'Bathilda Bagshot Magical': 'PERSON',
 'Beaters': 'GPE',
 'Beechwood': 'PERSON',
 'B

Named entity recognition with nltk entities and harry potter wikia categories

In [85]:
for entity in nltkEntities:
    if not(entity in wikiaEntities):
        try:
            found = wikia.search("harrypotter", entity, 1)
        except:
            found = []
        
        if len(found) > 0:
            describingSentence = nltk.sent_tokenize(wikia.page("harrypotter", found[0]).content)[0]
            pos_sent = nltk.pos_tag(nltk.word_tokenize(describingSentence))
            for e in nltk.RegexpParser("DS: {<VBZ|VBD><DT>?<JJ>*<NN|NNS><IN>?<DT>?<JJ>*<NN|NNS>*}").parse(pos_sent):
                if isinstance(e, nltk.tree.Tree):
                    text = " ".join([word for word, tag in e.leaves()[1:]])
                    wikiaEntities[entity] = text
                    break
                else:
                    wikiaEntities[entity] = "Unknown"
        else:
            wikiaEntities[entity] = "Unknown"

for e in wikiaEntities:
    print(e, end=': ')
    print(wikiaEntities[e])

FORIBIDDEN: Unknown
Father: the father of the young wizard
Harry's: a game
Chf: the first card of the Adventures
Neville Longbottom: Unknown
Mirror: Unknown
Eeylops Owl Emporium: a chain of shops
Elfric: Unknown
Gryffindor Seeker: a wizard
Stalagmite: a rocky formation
Fall: Unknown
Wish McGonagall: Unknown
Muggles: Unknown
Dursleys: Unknown
Don: Unknown
Haaaaaa: a student at
Gringotts: the only bank of the wizarding
Deep: a book about magical animals
Are: Unknown
Witchcraft: the British wizarding school
Grunnings: a drill-making company in
SLYTHERIN: Unknown
Cross Station: Unknown
Ronnie: a sports team
Daily: Unknown
Evil: Unknown
Fer Dumbledore: daily letters
Brazilian: Unknown
Draco: Unknown
Perks: a witch
Had Hagrid: the surname of a
Cauldron Cakes: a popular wizarding cake
MIRROR OF: Unknown
Great Humberto: a television programme
Neck: a hit love
Gred: Unknown
Mandy: a half-bloodwitch
Stonewall: Unknown
Good: a phrase
Barn: a piece of property
Abou: the surname of a French family


Named entity recognition with entity finding via grammar, classes via harry potter wikia

In [106]:
entities = {}
entity = []

for entry in pos_taged:
    if (entry[1].startswith("NN") or (entity and entry[1].startswith("IN"))):
        entity.append(entry)
    else:
        if (entity) and entity[-1][1].startswith("IN"):
            entity.pop()
        if (entity and " ".join(e[0] for e in entity)[0].isupper()):
            entities[" ".join(e[0] for e in entity)] = 1
        entity = []
        
    
for entity in entities:
    if not(entity in myWikiaEntities):
        try:
            found = wikia.search("harrypotter", entity, 1)
        except:
            found = []
        
        if len(found) > 0:
            try:
                sents_tok = nltk.sent_tokenize(wikia.page("harrypotter", found[0]).content)
            except:
                sents_tok = []
            if len(sents_tok) > 0:
                describingSentence = sents_tok[0]
                pos_sent = nltk.pos_tag(nltk.word_tokenize(describingSentence))
                for e in nltk.RegexpParser("DS: {<VBZ|VBD><DT>?<JJ>*<NN|NNS><IN>?<DT>?<JJ>*<NN|NNS>*}").parse(pos_sent):
                    if isinstance(e, nltk.tree.Tree):
                        text = " ".join([word for word, tag in e.leaves()[1:]])
                        myWikiaEntities[entity] = text
                        break
                    else:
                        myWikiaEntities[entity] = "Unknown"
            else:
                myWikiaEntities[entity] = "Unknown"
        else:
            myWikiaEntities[entity] = "Unknown"

for e in myWikiaEntities:
    print(e, end=': ')
    print(myWikiaEntities[e])

F: a witch
Father: the father of the young wizard
Harry's: a game
Grunnings parking lot: a drill-making company in
More'n: a wizard
Weasley sweater: a home-made jumper
Hopes: Unknown
Strict: Unknown
Dudley's: Unknown
Neville Longbottom: Unknown
So: Unknown
Eeylops Owl Emporium: a chain of shops
Uncle Algie: Unknown
Gryffindor Seeker: a wizard
Stalagmite: a rocky formation
Fall: Unknown
Wish McGonagall: Unknown
Muggles: Unknown
Harry as though he'd: Unknown
CHAPTER FIVE DIAGON ALLEY Harry: a cobblestoned wizarding alley
Dursleys: Unknown
Got: Unknown
Don: Unknown
Haaaaaa: a student at
Harry Potter day: Unknown
STUDENTS OUT OF BED: a list of students
SEIZE HIM: a charm
Deep: a book about magical animals
Fang: Unknown
Grunnings: a drill-making company in
Muggles in helicopters: a type of
SLYTHERIN: Unknown
Cross Station: Unknown
Make way: Unknown
Ronnie: a sports team
Stones that Snape: Unknown
Yesterday: Unknown
Jelly-Legs: a spell
Thanksss: a large brown snake
LOOK AT THIS SNAKE: a serp